## Import thư viện cần thiết

In [17]:
import numpy as np
import matplotlib.pyplot as plt
import chess
import chess.svg
from typing import List
import pandas as pd
from IPython.display import display, SVG, HTML
import math
from evaluation.evaluation import Evaluation
from evaluation.piece_square_table import PieceSquareTable
import sys
import os
import time
from bot import ChessBot



## Lấy tập dữ liệu

In [18]:
df = pd.read_csv('chessData.csv')
df.head()

,FEN,Evaluation
0,rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR ...,-10
1,rnbqkbnr/pppp1ppp/4p3/8/4P3/8/PPPP1PPP/RNBQKBN...,+56
2,rnbqkbnr/pppp1ppp/4p3/8/3PP3/8/PPP2PPP/RNBQKBN...,-9
3,rnbqkbnr/ppp2ppp/4p3/3p4/3PP3/8/PPP2PPP/RNBQKB...,+52
4,rnbqkbnr/ppp2ppp/4p3/3p4/3PP3/8/PPPN1PPP/R1BQK...,-26


In [19]:
sampled_df = df.sample(n=400, random_state=50)
test_fens = sampled_df['FEN'].to_list()

sampled_df_low = df.sample(n=80, random_state=70)
test_fens_low = sampled_df_low['FEN'].to_list()
sampled_df.to_csv("sample.csv")
sampled_df_low.to_csv("sample2.csv")


## Visualization

In [20]:
def showBoard(fens, boards_per_row=5):
    total_boards = len(fens)
    rows = math.ceil(total_boards / boards_per_row)
    
    for row in range(rows):
        boards_in_row = []
        for col in range(boards_per_row):
            idx = row * boards_per_row + col
            if idx < total_boards:
                board = chess.Board(fens[idx])
                svg = chess.svg.board(board, size=200)  # Giảm size nếu cần
                boards_in_row.append(SVG(svg)._repr_svg_())  # Lấy chuỗi SVG
        
        # Tạo HTML với flexbox để xếp hàng ngang
        html = f"""
        <div style="display: flex; flex-wrap: nowrap; gap: 10px; margin: 10px 0;">
            {''.join(boards_in_row)}
        </div>
        """
        display(HTML(html))





showBoard(df.sample(n=50, random_state=42)['FEN'].to_list())

# START CACULATE EXPANSION RATIO

In [21]:
total_time_by_depth = [0] * 15
expansion_ratio = [0] * 15

In [22]:
def total_time(depth:int, use_low: bool = False, nnue_order:bool = False) -> float:
    """hàm tìm hệ số mở rộng theo depth, để tối ưu thời gian với depth<=7 lấy tập dữ liệu rộng, với depth > 7  lấy tập hẹp"""
    #try:
    print(f"start caculate for depth: {depth}")
    current_fen = test_fens_low if use_low else test_fens
    for index, fen in enumerate(current_fen):
        start = time.perf_counter()
        stdout_goc = sys.stdout
        with open(os.devnull, 'w') as fnull:
            sys.stdout = fnull
            try:
                if nnue_order:
                    chess_bot = ChessBot(use_nnue=True)
                else:
                    chess_bot = ChessBot()
                chess_bot.set_position(fen)
                chess_bot.get_best_move(depth=depth)
            finally:
                sys.stdout = stdout_goc  # luôn khôi phục dù có lỗi
        end = time.perf_counter()
        total_time_by_depth[depth] += (end - start) * 1000
        print(f"✅ finish fen at index {index} at depth {depth}, total time: {total_time_by_depth[depth]:.2f} ms.")
        """except Exception as e:
        print("bug exist, return inf:", e)
        return float('inf')"""        
    print(f"total time at depth {depth} is {total_time_by_depth[depth]:.2f} ms")
    if depth > 1:
        ratio = total_time_by_depth[depth] / total_time_by_depth[depth-1]
        print(f"the expansion ratio at depth {depth} is {ratio:.3f}")
        return ratio
    return 0


In [23]:
for depth in range(1,11):
    if depth != 7:
        expansion_ratio[depth] = total_time(depth,(depth>7))
    else:
        expansion_ratio[depth] = total_time(depth,False) # lưu kết quả theo tập dữ liệu rộng
        total_time_by_depth[7] = 0
        total_time(depth,True) #tính toán để cho depth= 8


start caculate for depth: 1
✅ finish fen at index 0 at depth 1, total time: 15.61 ms.
✅ finish fen at index 1 at depth 1, total time: 33.66 ms.
✅ finish fen at index 2 at depth 1, total time: 102.96 ms.
✅ finish fen at index 3 at depth 1, total time: 149.96 ms.
✅ finish fen at index 4 at depth 1, total time: 284.10 ms.
✅ finish fen at index 5 at depth 1, total time: 349.17 ms.
✅ finish fen at index 6 at depth 1, total time: 373.20 ms.
✅ finish fen at index 7 at depth 1, total time: 439.87 ms.
✅ finish fen at index 8 at depth 1, total time: 474.22 ms.
✅ finish fen at index 9 at depth 1, total time: 533.22 ms.
✅ finish fen at index 10 at depth 1, total time: 641.13 ms.
✅ finish fen at index 11 at depth 1, total time: 690.16 ms.
✅ finish fen at index 12 at depth 1, total time: 804.39 ms.
✅ finish fen at index 13 at depth 1, total time: 818.78 ms.
✅ finish fen at index 14 at depth 1, total time: 866.08 ms.
✅ finish fen at index 15 at depth 1, total time: 883.83 ms.
✅ finish fen at index 16

Traceback (most recent call last):
  File "c:\Users\ADMIN\Downloads\code\chess\chess-bot\bot.py", line 221, in _search_thread
    self.searcher.start_search()
  File "c:\Users\ADMIN\Downloads\code\chess\chess-bot\search\searcher.py", line 87, in start_search
    print('finished search')
ValueError: write to closed file


✅ finish fen at index 129 at depth 4, total time: 337326.85 ms.
✅ finish fen at index 130 at depth 4, total time: 349526.69 ms.
✅ finish fen at index 131 at depth 4, total time: 353747.67 ms.
✅ finish fen at index 132 at depth 4, total time: 354575.18 ms.
✅ finish fen at index 133 at depth 4, total time: 359483.26 ms.
✅ finish fen at index 134 at depth 4, total time: 359735.09 ms.
✅ finish fen at index 135 at depth 4, total time: 366987.50 ms.
✅ finish fen at index 136 at depth 4, total time: 368384.49 ms.
✅ finish fen at index 137 at depth 4, total time: 369907.97 ms.
✅ finish fen at index 138 at depth 4, total time: 370932.74 ms.
✅ finish fen at index 139 at depth 4, total time: 371283.20 ms.
✅ finish fen at index 140 at depth 4, total time: 372774.09 ms.
✅ finish fen at index 141 at depth 4, total time: 373312.08 ms.
✅ finish fen at index 142 at depth 4, total time: 376866.54 ms.
✅ finish fen at index 143 at depth 4, total time: 379409.57 ms.
✅ finish fen at index 144 at depth 4, to

Traceback (most recent call last):
  File "c:\Users\ADMIN\Downloads\code\chess\chess-bot\bot.py", line 225, in _search_thread
    print(f"Search completed, best_move: {best_move}")
ValueError: write to closed file


Search completed, best_move: e5e6
✅ finish fen at index 281 at depth 6, total time: 4370922.24 ms.
✅ finish fen at index 282 at depth 6, total time: 4381680.73 ms.
finished search✅ finish fen at index 283 at depth 6, total time: 4411723.82 ms.

✅ finish fen at index 284 at depth 6, total time: 4432854.43 ms.
✅ finish fen at index 285 at depth 6, total time: 4462902.14 ms.
✅ finish fen at index 286 at depth 6, total time: 4486776.31 ms.
✅ finish fen at index 287 at depth 6, total time: 4487552.33 ms.
✅ finish fen at index 288 at depth 6, total time: 4492132.94 ms.
✅ finish fen at index 289 at depth 6, total time: 4496718.19 ms.
✅ finish fen at index 290 at depth 6, total time: 4503359.58 ms.
✅ finish fen at index 291 at depth 6, total time: 4506712.78 ms.
✅ finish fen at index 292 at depth 6, total time: 4517373.11 ms.
✅ finish fen at index 293 at depth 6, total time: 4521447.90 ms.
✅ finish fen at index 294 at depth 6, total time: 4535532.59 ms.
✅ finish fen at index 295 at depth 6, to

...

✅ finish fen at index 206 at depth 7, total time: 5096321.36 ms.
✅ finish fen at index 207 at depth 7, total time: 5105185.20 ms.
✅ finish fen at index 208 at depth 7, total time: 5135253.97 ms.
✅ finish fen at index 209 at depth 7, total time: 5165305.37 ms.
✅ finish fen at index 210 at depth 7, total time: 5195355.71 ms.
✅ finish fen at index 211 at depth 7, total time: 5225415.92 ms.
✅ finish fen at index 212 at depth 7, total time: 5255470.40 ms.
✅ finish fen at index 213 at depth 7, total time: 5282461.49 ms.
✅ finish fen at index 214 at depth 7, total time: 5293222.99 ms.
✅ finish fen at index 215 at depth 7, total time: 5323281.70 ms.
✅ finish fen at index 216 at depth 7, total time: 5353338.14 ms.finished search
g2h1
Search completed, best_move: g2h1

✅ finish fen at index 217 at depth 7, total time: 5383395.10 ms.
✅ finish fen at index 218 at depth 7, total time: 5413452.18 ms.
✅ finish fen at index 219 at depth 7, total time: 5443500.70 ms.
✅ finish fen at index 220 at depth 

Traceback (most recent call last):
  File "c:\Users\ADMIN\Downloads\code\chess\chess-bot\bot.py", line 221, in _search_thread
    self.searcher.start_search()
  File "c:\Users\ADMIN\Downloads\code\chess\chess-bot\search\searcher.py", line 87, in start_search
    print('finished search')
ValueError: write to closed file


✅ finish fen at index 55 at depth 7, total time: 1436284.39 ms.
✅ finish fen at index 56 at depth 7, total time: 1466318.00 ms.
✅ finish fen at index 57 at depth 7, total time: 1474573.87 ms.
✅ finish fen at index 58 at depth 7, total time: 1504601.91 ms.
✅ finish fen at index 59 at depth 7, total time: 1534649.04 ms.
✅ finish fen at index 60 at depth 7, total time: 1564715.95 ms.
✅ finish fen at index 61 at depth 7, total time: 1594748.77 ms.
✅ finish fen at index 62 at depth 7, total time: 1624791.65 ms.
✅ finish fen at index 63 at depth 7, total time: 1654824.66 ms.
✅ finish fen at index 64 at depth 7, total time: 1684871.87 ms.
✅ finish fen at index 65 at depth 7, total time: 1714915.95 ms.
✅ finish fen at index 66 at depth 7, total time: 1744959.52 ms.
✅ finish fen at index 67 at depth 7, total time: 1774997.07 ms.
✅ finish fen at index 68 at depth 7, total time: 1805040.54 ms.
✅ finish fen at index 69 at depth 7, total time: 1835096.06 ms.
✅ finish fen at index 70 at depth 7, tot

Traceback (most recent call last):
  File "c:\Users\ADMIN\Downloads\code\chess\chess-bot\bot.py", line 225, in _search_thread
    print(f"Search completed, best_move: {best_move}")
ValueError: write to closed file


✅ finish fen at index 23 at depth 10, total time: 898273.42 ms.
✅ finish fen at index 24 at depth 10, total time: 928322.29 ms.
✅ finish fen at index 25 at depth 10, total time: 958367.03 ms.
✅ finish fen at index 26 at depth 10, total time: 988436.97 ms.
✅ finish fen at index 27 at depth 10, total time: 1018496.89 ms.finished search
b6a4
Search completed, best_move: b6a4

✅ finish fen at index 28 at depth 10, total time: 1048539.14 ms.
✅ finish fen at index 29 at depth 10, total time: 1078598.07 ms.
✅ finish fen at index 30 at depth 10, total time: 1108641.94 ms.
✅ finish fen at index 31 at depth 10, total time: 1138691.43 ms.
✅ finish fen at index 32 at depth 10, total time: 1168744.89 ms.
✅ finish fen at index 33 at depth 10, total time: 1198780.90 ms.
Search completed, best_move: g3f4
✅ finish fen at index 34 at depth 10, total time: 1228839.87 ms.
✅ finish fen at index 35 at depth 10, total time: 1258894.42 ms.
✅ finish fen at index 36 at depth 10, total time: 1288950.09 ms.
e6f4


In [24]:
for depth in range(1,11):
    print(f"the expansion ratio at depth {depth} is {expansion_ratio[depth]}")

the expansion ratio at depth 1 is 0
the expansion ratio at depth 2 is 5.918685130964094
the expansion ratio at depth 3 is 2.31559020535454
the expansion ratio at depth 4 is 2.928389878801775
the expansion ratio at depth 5 is 2.977312373936534
the expansion ratio at depth 6 is 2.206598636221307
the expansion ratio at depth 7 is 1.49191046769435
the expansion ratio at depth 8 is 1.0995220641809031
the expansion ratio at depth 9 is 1.0128484938632873
the expansion ratio at depth 10 is 1.086859457633767
